# Reading and writing LAS files

This notebook goes with [the Agile blog post](https://agilescientific.com/blog/2017/10/23/x-lines-of-python-load-curves-from-las) of 23 October 2017.

You'll need `welly` in your environment:

    pip install welly
    
This will also install the latest versions of `striplog` and `lasio`.

In [ ]:
ls ../data/*.[Ll][Aa][Ss]

## Load the LAS file with `lasio`

In [ ]:
import lasio

l = lasio.read('../data/P-129.LAS')

That's it! But the object itself doesn't tell us much — it's really just a container:

In [ ]:
l.header

In [ ]:
l.curves

## Look at the WELL section of the header

In [ ]:
l.header['Well']

## Look at the curve data

The curves are all present one big NumPy array:

In [ ]:
l.data

In [ ]:
l.curves

Or we can go after a single curve object:

In [ ]:
l.curves.GR.data

And there's a shortcut to its data:

In [ ]:
l['GR']

...so it's easy to make a plot against depth:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(15,3))
plt.plot(l['DEPT'], l['GR'])
plt.show()

## Inspect the curves as a `pandas` dataframe

In [ ]:
l.df().head()

In [ ]:
plt.plot(l.df().GR)

## Load the LAS file with `welly` 

`welly` uses `lasio` to load the file. Then it adds some handy things:

- Richer `curve` objects (subclassed NumPy arrays), with smoothing, resampling, plotting, and more.
- Curve alias lists.
- Curve quality checking.
- Remapping and transforming fields during LAS load.
- A `project` object for managing several wells at once.
- A `well.location` property, with a CRS and directional survey.
- Jupyter Notebook friendly object representations.

In [ ]:
from welly import Well

w = Well.from_las('../data/P-129.LAS')

`welly` Wells know how to display some basics:

In [ ]:
w

We can plot all the curves in a well:

In [ ]:
w.plot()

And the `Well` object also has `lasio`'s access to a pandas DataFrame:

In [ ]:
w.df().head()

## Look at `welly`'s Curve object

The `data` attribute is just a dictionary:

In [ ]:
w.data

Like the `Well`, a `Curve` object can report a bit about itself:

In [ ]:
gr = w.data['GR']
gr

To get at the depth in a `lasio` object, you have to find the `DEPT` curve:

In [ ]:
w.las.curves.DEPT

One important thing about `Curve` objects is that each one knows its own depths — they are stored as a property called `basis`. (It's not actually stored, but computed on demand from the start depth, the sample interval (which must be constant for the whole curve) and the number of samples in the object.)

In [ ]:
gr.basis

In [ ]:
gr.plot()

## Plot part of a curve

We'll grab the interval from 300 m to 1000 m and plot it.

In [ ]:
gr.to_basis(start=300, stop=1000).plot()  # Line 8.

In [ ]:
gr.to_basis(start=500, stop=510, step=0.1).plot(color='r')

## Curve quality

We can run qualaity tests on wells or on projects.

In [ ]:
from IPython.display import HTML
import welly.quality as qc

tests = {
    'each': [qc.no_gaps, qc.no_monotonic],
    'GR': [qc.all_positive]
}

results = w.qc_table_html(tests=tests)

HTML(results)

## Smooth a curve

Curve objects are, fundamentally, NumPy arrays. But they have some extra tricks. We've already seen `Curve.plot()`. 

Using the `Curve.smooth()` method, we can easily smooth a curve, eg by 15 m (passing `samples=True` would smooth by 15 samples):

In [ ]:
sm = gr.smooth(window_length=15, samples=False)

fig, ax = plt.subplots(figsize=(2,10))
ax = gr.plot(ax=ax)
ax = sm.plot(ax=ax, lw=2)

We can save this as a new LAS file:

In [ ]:
x = Well(params={'name': w.header.name})
x.header = w.header
x.location = w.location

In [ ]:
x.data['GR'] = w.data['GR']

x.data['GR_smooth'] = sm

x.data

In [ ]:
x.to_las('../data/x.las')

## Export a set of curves as a matrix

`welly` lets you specify which curves you want (optionally using aliases like GR1, GRC, NGC, etc for GR), resample the data, or specify a start and stop depth. This method is also wrapped by `Project.data_as_matrix()` which is nice because it ensures that all the wells are exported at the same sample interval.

Here are the curves in this well:

In [ ]:
w.data.keys()

Let's say we only want to export a few curves for our machine learning project:

In [ ]:
keys = ['CALI', 'DT', 'DTS', 'RHOB', 'SP']

In [ ]:
w.plot(tracks=['TVD']+keys)

In [ ]:
X, basis = w.data_as_matrix(keys=keys, start=275, stop=1850, step=0.5, return_basis=True)

In [ ]:
w.data['CALI'].shape

So CALI had 12,718 points in it... since we downsampled to 0.5 m and removed the top and tail, we should have substantially fewer points:

In [ ]:
X.shape

In [ ]:
plt.figure(figsize=(15,3))
plt.plot(X.T[0])
plt.show()

## The `Project` class

We can also load multiple wells into a Welly `Project`:

In [ ]:
from welly import Project

p = Project.from_las('../data/*.las')

In [ ]:
p

Projects are really just lists of wells:

In [ ]:
p[4]

We can make a quality table for all the wells in the project.

It helps to use an alias dictionary for this, becase otherwise there will be a lot of curves.

We'll use the same tests as before.

In [ ]:
from welly.defaults import ALIAS

HTML(p.curve_table_html(tests=tests, alias=ALIAS, keys=['DT', 'GR', 'CAL']))

## BONUS: fix the lat, lon

**Warning:** This is pretty gross. But you're going to need it sooner or later...

Did you notice that the location of the well did not get loaded properly?

In [ ]:
w.uwi

In [ ]:
w.location

Let's look at some of the header:

    # LAS format log file from PETREL
    # Project units are specified as depth units
    #==================================================================
    ~Version information
    VERS.   2.0:
    WRAP.   YES:
    #==================================================================
    ~WELL INFORMATION
    #MNEM.UNIT      DATA             DESCRIPTION
    #---- ------ --------------   -----------------------------
    STRT .M      1.0668          :START DEPTH     
    STOP .M      1939.13760      :STOP DEPTH     
    STEP .M       0.15240        :STEP        
    NULL .          -999.25      :NULL VALUE
    COMP .        Elmworth Energy Corporation              :COMPANY
    WELL .        Kennetcook #2                            :WELL
    FLD  .        Windsor Block                            :FIELD
    LOC  .        Lat = 45* 12' 34.237" N                  :LOCATION
    PROV .        Nova Scotia                              :PROVINCE
      UWI.        Long = 63* 45'24.460  W                  :UNIQUE WELL ID
    LIC  .        P-129                                    :LICENSE NUMBER
    CTRY .        CA                                       :COUNTRY (WWW code)
     DATE.        10-Oct-2007                              :LOG DATE {DD-MMM-YYYY}
    SRVC .        Schlumberger                             :SERVICE COMPANY
    LATI .DEG                                              :LATITUDE
    LONG .DEG                                              :LONGITUDE
    GDAT .                                                 :GeoDetic Datum
    SECT .        45.20 Deg N                              :Section
    RANG .        PD 176                                   :Range
    TOWN .        63.75 Deg W                              :Township

Look at **LOC** and **UWI**. There are two problems:

1. These items are in the wrong place. (Notice **LATI** and **LONG** are empty.)
2. The items are malformed, with lots of extraneous characters.

We can fix this in two steps:

1. Remap the header items to fix the first problem.
2. Parse the items to fix the second one.

We'll define these in reverse because the remapping uses the transforming function.

In [ ]:
import re
import welly

def transform_ll(text):
    """
    Parses malformed lat and lon so they load properly.
    """
    def callback(match):
        d = match.group(1).strip()
        m = match.group(2).strip()
        s = match.group(3).strip()
        c = match.group(4).strip()
        if c.lower() in ('w', 's') and d[0] != '-':
            d = '-' + d
        return ' '.join([d, m, s])
    pattern = re.compile(r""".+?([-0-9]+?).? ?([0-9]+?).? ?([\.0-9]+?).? +?([NESW])""", re.I)
    text = pattern.sub(callback, text)
    return welly.utils.dms2dd([float(i) for i in text.split()])

Make sure that works!

In [ ]:
print(transform_ll("""Lat = 45* 12' 34.237" N"""))

It does!

Now we can define two dictionaries:

- `remap` tells welly what to use for certain fields when loading the well. The first item says, "Use whatever is in the LOC parameter for the LATI parameter"... or "Get LATI from LOC". The last item says, "Don't load the parameter TOWN from this file" and it will thus be ignored.

- `funcs` tells welly what functions to run on a field as it is loaded. Note that if we remap a field, you specify the field you are mapping TO (that is, the target field name) as the key. The first item in the `funcs` dict, below, tells welly to pass the parameter through the function `transform_ll` during loading.

In [ ]:
remap = {
    'LATI': 'LOC',  # Use LOC for the parameter LATI.
    'LONG': 'UWI',  # Use UWI for the parameter LONG.
    'LOC':  None,   # Use nothing for the parameter SECT.
    'SECT': None,   # Use nothing for the parameter SECT.
    'RANG': None,   # Use nothing for the parameter RANG.
    'TOWN': None,   # Use nothing for the parameter TOWN.
}

funcs = {
    'LATI': transform_ll,  # Pass LATI through this function before loading.
    'LONG': transform_ll,  # Pass LONG through it too.
    'UWI': lambda x: "No UWI, fix this!"
}

Now we can load the file again, this time with the `remap` and `funcs`:

In [ ]:
w = Well.from_las('../data/P-129.LAS', remap=remap, funcs=funcs)

Let's see if it fixed the problem:

In [ ]:
w.location.latitude, w.location.longitude

In [ ]:
w.location

In [ ]:
w.uwi

Yes!

Let's just hope the mess is the same mess in every well. (LOL, no-one's that lucky.)

<hr>

**&copy; 2017 [agilescientific.com](https://www.agilescientific.com/) and licensed [CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/)**